In [2]:
using JuMP
using ConditionalJuMP
using Gurobi

INFO: Recompiling stale cache file /home/rdeits/.julia/lib/v0.6/ConditionalJuMP.ji for module ConditionalJuMP.


In [10]:
Δt_bounds = (0, 0.1)
f_bounds = (0, 20)
q_bounds = (0, 10)
v_bounds = (-100, 100)
mass = 1.0
g = -9.81

struct Result
    x
    Δt
    fΔt
end

JuMP.getvalue(r::Result) = Result(getvalue(r.x), getvalue(r.Δt), getvalue(r.fΔt))

function update(x, model::Model)
    Δt = @variable(model, basename="Δt",
        lowerbound=Δt_bounds[1], upperbound=Δt_bounds[2])
    fΔt = @variable(model, basename="fΔt",
        lowerbound=f_bounds[1] * Δt_bounds[1], upperbound=f_bounds[2] * Δt_bounds[2])
    qnext = @variable(model, basename="q",
        lowerbound=q_bounds[1], upperbound=q_bounds[2])
    vnext = @variable(model, basename="v",
        lowerbound=v_bounds[1], upperbound=v_bounds[2])
    
    @disjunction(model, (qnext == 0), (fΔt == 0))
    
    total_impulse = fΔt + mass * g * Δt
    
    @constraints model begin
        (qnext - x[1]) == vnext * Δt # problem is here, since vnext and dt are decision variables
        mass * (vnext - x[2]) == total_impulse
    end
    Result([qnext, vnext], Δt, fΔt)
end
    
function optimize(x0, N, model=Model(solver=GurobiSolver(OutputFlag=0)))
    results = []
    x = x0
    for i in 1:N
        push!(results, update(x, model))
        x = results[end].x
    end
    results, model
end

optimize (generic function with 2 methods)

In [ ]:
x0 = [1.0, 0.0]

results, model = optimize(x0, 20)
solve(model)